# standard

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from pyzzle import Puzzle, FancyPuzzle, Dictionary, ObjectiveFunction, Optimizer, utils, Mask

%load_ext autoreload
%autoreload 2

In [4]:
# puzzle = FancyPuzzle(Mask.donut_s)
puzzle = Puzzle(35,40)
puzzle.import_dict(Dictionary.dataset.shanghai_exe)

In [16]:
puzzle.first_solve()

In [17]:
puzzle.show()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,ポ,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,カ,レ,ハ,ウ,ェ,イ,ブ,,,パ,ネ,ル,シ,ュ,ー,ト,,,,,,,レ,,,サ
2,,,ク,ッ,サ,ク,ド,リ,ル,,,,,,ズ,,,,ワ,,,,,,,,,,チ,ャ,ー,ジ,カ,ノ,ン
3,,フ,,,,イ,,,,,,,ブ,ロ,ン,ズ,ナ,パ,ー,ム,,,,,,,ポ,,,,,ス,,,ダ
4,,ァ,,グ,,ッ,,サ,,,,フ,,,ロ,,,,オ,,,,,,パ,ラ,イ,ズ,カ,セ,ッ,ト,,,ー
5,,イ,,ラ,,ク,,ン,,リ,フ,レ,ス,テ,ー,ジ,,,ー,,,,,,,,ズ,,,,,,,,ロ
6,,ア,,ン,,カ,,ダ,,,,イ,,,ド,,,,ラ,,エ,レ,キ,チ,ェ,ー,ン,,,,,,,,ー
7,,ネ,,ド,,ス,ロ,ー,カ,ス,タ,ム,,,,,,,,,ル,,,,,,ス,,ハ,,,,,,ド
8,ネ,ッ,プ,ウ,,,,ス,,ナ,,ア,,,,,,,,,ジ,,,ギ,,,テ,,ク,,,,,カ,
9,,ト,,ェ,,ゴ,,テ,,ジ,,ー,,,フ,,グ,ラ,ス,ロ,ー,ド,,ガ,,,ー,,ロ,,,,,ス,


In [33]:
obj_func = ["nwords", "cross_count", "fill_count", "max_connected_empties"])
puzzle.solve(epoch=20, optimizer="local_search", of=obj_func)

>>> Interim solution


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,ポ,,,,,,,,,ポ,イ,ズ,ン,ス,テ,ー,ジ,,,,パ
1,,,,,,,,,,カ,レ,ハ,ウ,ェ,イ,ブ,,,パ,,,,,イ,,,,,,,,,,,ネ
2,,メ,タ,リ,ッ,ク,オ,ー,ラ,,,,,,ズ,,,,ワ,,,,,ズ,,,ツ,,,ゼ,ロ,ナ,ッ,ク,ル
3,,,,,,イ,,,,,,,ブ,ロ,ン,ズ,ナ,パ,ー,ム,,,ポ,ン,ポ,コ,ジ,ゾ,ウ,,,,,,シ
4,フ,,,グ,,ッ,,サ,,,,フ,,,ロ,,,,オ,,,,,ネ,,,ギ,,,,,,,,ュ
5,ァ,,,ラ,,ク,,ン,,リ,フ,レ,ス,テ,ー,ジ,,コ,ー,ル,ド,シ,ョ,ッ,ト,,リ,,,グ,,,,,ー
6,イ,,,ン,,カ,,ダ,,,,イ,,,ド,,,,ラ,,,,,ト,,,ソ,,ト,ラ,ッ,プ,ネ,ッ,ト
7,ア,,,ド,,ス,ロ,ー,カ,ス,タ,ム,,,,,,,,,,,,,,,ー,,,ス,,,,,
8,ネ,ッ,プ,ウ,,,,ス,,ナ,,ア,,,,,,,,,,グ,ラ,ス,ロ,ー,ド,,,ス,,,,カ,
9,ッ,,,ェ,,ゴ,,テ,,ジ,,ー,,,フ,,,,,,,,,,,,,,,テ,,レ,ジ,ス,ト


>>> Epoch 131/150
    - Stayed: [ 111.  111.  605. 1264.]
>>> Epoch 132/150
    - Stayed: [ 111.  111.  605. 1264.]
>>> Epoch 133/150
    - Replaced(same score): [ 111.  111.  605. 1264.] -> [ 111.  111.  605. 1264.]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,ポ,,,,,,,,,ポ,イ,ズ,ン,ス,テ,ー,ジ,,,,パ
1,,,,,,,,,,カ,レ,ハ,ウ,ェ,イ,ブ,,,パ,,,,,イ,,,,,,,,,,,ネ
2,,メ,タ,リ,ッ,ク,オ,ー,ラ,,,,,,ズ,,,,ワ,,,,,ズ,,,ツ,,,ゼ,ロ,ナ,ッ,ク,ル
3,,,,,,イ,,,,,,,ブ,ロ,ン,ズ,ナ,パ,ー,ム,,,ポ,ン,ポ,コ,ジ,ゾ,ウ,,,,,,シ
4,フ,,,グ,,ッ,,サ,,,,フ,,,ロ,,,,オ,,,,,ネ,,,ギ,,,,,,,,ュ
5,ァ,,,ラ,,ク,,ン,,リ,フ,レ,ス,テ,ー,ジ,,コ,ー,ル,ド,シ,ョ,ッ,ト,,リ,,,グ,,,,,ー
6,イ,,,ン,,カ,,ダ,,,,イ,,,ド,,,,ラ,,,,,ト,,,ソ,,ト,ラ,ッ,プ,ネ,ッ,ト
7,ア,,,ド,,ス,ロ,ー,カ,ス,タ,ム,,,,,,,,,,,,,,,ー,,,ス,,,,,
8,ネ,ッ,プ,ウ,,,,ス,,ナ,,ア,,,,,,,,,,グ,ラ,ス,ロ,ー,ド,,,ス,,,,カ,
9,ッ,,,ェ,,ゴ,,テ,,ジ,,ー,,,フ,,,,,,,,,,,,,,,テ,,レ,ジ,ス,ト


>>> Epoch 134/150
    - Stayed: [ 111.  111.  605. 1264.]
>>> Epoch 135/150
    - Improved: [ 111.  111.  605. 1264.] --> [ 111.  111.  608. 1266.]


...

>>> Epoch 136/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 137/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 138/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 139/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 140/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 141/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 142/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 143/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 144/150
    - Replaced(same score): [ 111.  111.  608. 1266.] -> [ 111.  111.  608. 1266.]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,ポ,,,,,,,,,ポ,イ,ズ,ン,ス,テ,ー,ジ,,,,パ
1,,,,,,,,,,カ,レ,ハ,ウ,ェ,イ,ブ,,,パ,,,,,イ,,,,,,,,,ネ,,ネ
2,,メ,タ,リ,ッ,ク,オ,ー,ラ,,,,,,ズ,,,,ワ,,,,,ズ,,,ツ,,,ゼ,ロ,ナ,ッ,ク,ル
3,,ル,,,,イ,,,,,,,ブ,ロ,ン,ズ,ナ,パ,ー,ム,,,ポ,ン,ポ,コ,ジ,ゾ,ウ,,,,プ,,シ
4,,テ,,,,ッ,,サ,,,,フ,,,ロ,,,,オ,,,,,ネ,,,ギ,,,,,,ウ,,ュ
5,,ィ,,モ,,ク,,ン,,リ,フ,レ,ス,テ,ー,ジ,,コ,ー,ル,ド,シ,ョ,ッ,ト,,リ,,,グ,,,,,ー
6,,ン,,ン,,カ,,ダ,,,,イ,,,ド,,,,ラ,,,,,ト,,,ソ,,ト,ラ,ッ,プ,ネ,ッ,ト
7,,グ,,キ,,ス,ロ,ー,カ,ス,タ,ム,,,,,,,,,,,,,,,ー,,,ス,,,,,
8,,ロ,,ー,,,,ス,,ナ,,ア,,,,,,,,,,グ,ラ,ス,ロ,ー,ド,,,ス,,,,カ,
9,,ウ,,ポ,,ゴ,,テ,,ジ,,ー,,,フ,,,,,,,,,,,,,,,テ,,レ,ジ,ス,ト


>>> Epoch 145/150
    - Stayed: [ 111.  111.  608. 1266.]
>>> Epoch 146/150
    - Improved: [ 111.  111.  608. 1266.] --> [ 111.  111.  609. 1267.]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,ポ,,,,,,,,,ポ,イ,ズ,ン,ス,テ,ー,ジ,,,,パ
1,,,,,,,,,,カ,レ,ハ,ウ,ェ,イ,ブ,,,パ,,,,,イ,,,,,,,,,ネ,,ネ
2,,メ,タ,リ,ッ,ク,オ,ー,ラ,,,,,,ズ,,,,ワ,,,,,ズ,,,ツ,,,ゼ,ロ,ナ,ッ,ク,ル
3,,ル,,,,イ,,,,,,,ブ,ロ,ン,ズ,ナ,パ,ー,ム,,,ポ,ン,ポ,コ,ジ,ゾ,ウ,,,,プ,,シ
4,,テ,,,,ッ,,サ,,,,フ,,,ロ,,,,オ,,,,,ネ,,,ギ,,,,,,ウ,,ュ
5,,ィ,,モ,,ク,,ン,,リ,フ,レ,ス,テ,ー,ジ,,コ,ー,ル,ド,シ,ョ,ッ,ト,,リ,,,グ,,,,,ー
6,,ン,,ン,,カ,,ダ,,,,イ,,,ド,,,,ラ,,,,,ト,,,ソ,,ト,ラ,ッ,プ,ネ,ッ,ト
7,,グ,,キ,,ス,ロ,ー,カ,ス,タ,ム,,,,,,,,,,,,,,,ー,,,ス,,,,,
8,,ロ,,ー,,,,ス,,ナ,,ア,,,,,,,,,ホ,ワ,イ,ト,カ,ー,ド,,,ス,,,,カ,
9,,ウ,,ポ,,ゴ,,テ,,ジ,,ー,,,フ,,,,,,,,,,,,,,,テ,,レ,ジ,ス,ト


>>> Epoch 147/150
    - Stayed: [ 111.  111.  609. 1267.]
>>> Epoch 148/150
    - Stayed: [ 111.  111.  609. 1267.]
>>> Epoch 149/150
    - Stayed: [ 111.  111.  609. 1267.]
>>> Epoch 150/150
    - Stayed: [ 111.  111.  609. 1267.]


In [25]:
puzzle.save_answer_image("ep50.png")

In [ ]:
enable_and_cell = np.full(puzzle.cell.shape, " ")
enable_and_cell[puzzle.cover>0] = puzzle.cell[puzzle.cover>0]
enable_and_cell[puzzle.enable==False] = "×"
enable_and_cell[puzzle.mask==False] = "■"
utils.show_2Darray(enable_and_cell)

In [45]:
jj, ii = np.meshgrid(np.arange(puzzle.width), np.arange(puzzle.height))
# 縦
head0 = (puzzle.cell[ii[0,:], jj[0,:]] != "") * (puzzle.cell[ii[0,:]+1, jj[0,:]] != "")
body0 = (puzzle.cell[ii[1:-1,:]-1, jj[1:-1,:]] == "") * (puzzle.cell[ii[1:-1,:], jj[1:-1,:]] != "") * (puzzle.cell[ii[1:-1,:]+1, jj[1:-1,:]] != "")
start0 = np.vstack([head0, body0])

# 横
head1 = (puzzle.cell[ii[:,0], jj[:,0]] != "") * (puzzle.cell[ii[:,0], jj[:,0]+1] != "")
body1 = (puzzle.cell[ii[:,1:-1], jj[:,1:-1]-1] == "") * (puzzle.cell[ii[:,1:-1], jj[:,1:-1]] != "") * (puzzle.cell[ii[:,1:-1], jj[:,1:-1]+1] != "")
start1 = np.hstack([head1.reshape(puzzle.height,1), body1])

indices = {"vertical":np.where(start0), "horizontal":np.where(start1)}

used_words = []
enable = np.ones(puzzle.cell.shape).astype(bool)
for i, j in zip(indices["vertical"][0], indices["vertical"][1]):
    # used_words
    try:
        imax = i + np.where(puzzle.cell[i:, j]=='')[0][0]
    except:
        imax = puzzle.height
    used_words.append(''.join(puzzle.cell[i:imax,j]))
    # enable
    if i != 0:
        enable[i-1,j] = False
    if imax != puzzle.height:
        enable[imax,j] = False
    
for i, j in zip(indices["horizontal"][0], indices["horizontal"][1]):
    # used_words
    try:
        jmax = j + np.where(puzzle.cell[i, j:]=='')[0][0]
    except:
        jmax = puzzle.width
    used_words.append(''.join(puzzle.cell[i,j:jmax]))
    # enable
    if j != 0:
        enable[i,j-1] = False
    if jmax != puzzle.height:
        enable[i,jmax] = False